In [2]:
import warnings
import unionml

from pictionary_app import model

warnings.simplefilter("ignore")

In [3]:
num_classes = 3

# run training on a remote
execution = model.remote_train(
    hyperparameters={"num_classes": num_classes},
    trainer_kwargs={"num_epochs": 1},
    data_dir="./data",
    max_examples_per_class=1000,
    class_limit=num_classes,
)

Executing quickdraw_classifier.train, execution name: f475932fceb88435f94c.
Go to https://playground.hosted.unionai.cloud/console/projects/unionml/domains/development/executions/f475932fceb88435f94c to see the execution in the console.


In [4]:
# wait for the execution to complete, then a
model.remote_load(execution)

Waiting for execution f475932fceb88435f94c to complete...
Done.


In [5]:
import gradio as gr

gr.Interface(
    fn=model.predict,
    inputs="sketchpad",
    outputs="label",
    live=True,
    allow_flagging="never",
).launch()

Running on local URL:  http://127.0.0.1:7860/

To create a public link, set `share=True` in `launch()`.


(<fastapi.applications.FastAPI at 0x7faada1be100>,
 'http://127.0.0.1:7860/',
 None)

In [ ]:
# - get the features and labels locally
# - get model.artifact.model_object
# - invoke the Flyte workflow for predictions
# - visualize the original images, with accuracy score

In [7]:
from pictionary_app.dataset import QuickDrawDataset

dataset = QuickDrawDataset("/tmp/quickdraw_data", max_items_per_class=1000, class_limit=num_classes)
dataset

  0%|          | 0/3 [00:00<?, ?it/s]

Loading 1000 examples for each class from the Quickdraw Dataset...


  0%|          | 0/3 [00:00<?, ?it/s]

In [8]:
model.artifact.model_object

Sequential(
  (0): Conv2d(1, 64, kernel_size=(3, 3), stride=(1, 1), padding=same)
  (1): ReLU()
  (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (3): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=same)
  (4): ReLU()
  (5): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (6): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=same)
  (7): ReLU()
  (8): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (9): Flatten(start_dim=1, end_dim=-1)
  (10): Linear(in_features=2304, out_features=512, bias=True)
  (11): ReLU()
  (12): Linear(in_features=512, out_features=3, bias=True)
)

Create a `FlyteRemote` object and fetch the batch prediction workflow

In [11]:
from flytekit.remote import FlyteRemote
from flytekit.configuration import Config


remote = FlyteRemote(config=Config.auto(config_file="./config/config-remote.yaml"))
wf = remote.fetch_workflow(project="unionml", domain="development", name="flytekit_demo.batch_predictions.wf", version="v1")

Get features and labels

In [21]:
features = []
labels = []
for i in range(5):
    X, y = dataset[i]
    features.append(X.squeeze())
    labels.append(y)

In [22]:
execution = remote.execute(
    wf,
    inputs={"model_object": model.artifact.model_object, "features": features},
    project="unionml",
    domain="development",
    wait=True,
)

{"asctime": "2022-04-27 18:26:34,765", "name": "flytekit", "levelname": "ERROR", "message": "Exception when trying to execute ['aws', 's3', 'cp', '--acl', 'bucket-owner-full-control', '/var/folders/4q/frdnh9l10h53gggw1m59gr9m0000gn/T/flytexu3nt0_r/control_plane_metadata/local_flytekit/991eb3068ec048aaa6dadc47f92c95b8', 's3://my-s3-bucket/data/d9a39eab0e9cf865f9178a121b04ea60/991eb3068ec048aaa6dadc47f92c95b8'], reason: Called process exited with error code: 1.  Stderr dump:\n\nb'upload failed: ../../../../../var/folders/4q/frdnh9l10h53gggw1m59gr9m0000gn/T/flytexu3nt0_r/control_plane_metadata/local_flytekit/991eb3068ec048aaa6dadc47f92c95b8 to s3://my-s3-bucket/data/d9a39eab0e9cf865f9178a121b04ea60/991eb3068ec048aaa6dadc47f92c95b8 An error occurred (AccessDenied) when calling the PutObject operation: Access Denied\\n'"}
{"asctime": "2022-04-27 18:26:41,368", "name": "flytekit", "levelname": "ERROR", "message": "Exception when trying to execute ['aws', 's3', 'cp', '--acl', 'bucket-owner-fu

FlyteAssertion: Failed to put data from /var/folders/4q/frdnh9l10h53gggw1m59gr9m0000gn/T/flytexu3nt0_r/control_plane_metadata/local_flytekit/991eb3068ec048aaa6dadc47f92c95b8 to s3://my-s3-bucket/data/d9a39eab0e9cf865f9178a121b04ea60/991eb3068ec048aaa6dadc47f92c95b8 (recursive=False).

Original exception: Called process exited with error code: 1.  Stderr dump:

b'upload failed: ../../../../../var/folders/4q/frdnh9l10h53gggw1m59gr9m0000gn/T/flytexu3nt0_r/control_plane_metadata/local_flytekit/991eb3068ec048aaa6dadc47f92c95b8 to s3://my-s3-bucket/data/d9a39eab0e9cf865f9178a121b04ea60/991eb3068ec048aaa6dadc47f92c95b8 An error occurred (AccessDenied) when calling the PutObject operation: Access Denied\n'